In [1]:
# PET scans 

In [2]:
import numpy as np
import pandas as pd
import os

import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

from neuromaps.images import load_data, load_gifti, annot_to_gifti, relabel_gifti, construct_shape_gii
from neuromaps.datasets import fetch_annotation
from neuromaps.resampling import resample_images
from neuromaps.nulls import alexander_bloch, burt2020
from neuromaps.parcellate import Parcellater
from scipy.stats import pearsonr
from neuromaps import transforms 
from neuromaps.stats import compare_images
from neuromaps.nulls import hungarian

In [3]:
annotations = list(fetch_annotation(source=['hillmer2016',
                                            'tuominen',
                                            'naganawa2020']).keys())

In [4]:
annotations

[('hillmer2016', 'flubatine', 'MNI152', '1mm'),
 ('naganawa2020', 'lsn3172176', 'MNI152', '1mm'),
 ('tuominen', 'feobv', 'MNI152', '2mm')]

In [5]:
# define working dir 
base_path = os.path.dirname(os.getcwd())

# create a object that will parcellate data into dk atlas
dk_mni = os.path.join(base_path, 'parcellations', 'atlas-desikankilliany_space-MNI_res-1mm.nii.gz')
parcellater_mni = Parcellater(dk_mni , 'MNI152')

In [20]:
# parcellate FEOBV data 
smoking_pctdiff_file = os.path.join(base_path, 'data', 'smoking_pctdiff.nii.gz')
smoking_parc = parcellater_mni.fit_transform(smoking_pctdiff_file, space='MNI152', ignore_background_data=True)


labels = pd.read_csv(os.path.join(base_path, 'parcellations','atlas-desikankilliany.csv') )
labels['pctdiff'] = smoking_parc[0]
labels = labels[labels['structure'] == 'cortex']
labels

smoking_parc = labels['pctdiff']

In [21]:
smoking_parc

0     5.062069
1     1.545742
2     5.585381
3     7.520157
4    -0.441212
        ...   
70    6.541697
71    3.729597
72    4.260940
73    3.277341
74    2.641653
Name: pctdiff, Length: 68, dtype: float64

In [22]:
# download the regions for MNI152, take indecies of surface rois  
rois = pd.read_csv(os.path.join(base_path, 'parcellations' ,'atlas-desikankilliany.csv'))
rois = rois[(rois['structure'] == 'cortex')].index.to_numpy()

parcellated = dict([])
for (src, desc, space, den) in annotations:
    annot = fetch_annotation(source=src, desc=desc, space=space, den=den)
    parcellated[desc] = parcellater_mni.fit_transform(annot, space=space, ignore_background_data=True)

    # if subcortex included remove 
    if parcellated[desc].shape == (1,83):
        parcellated[desc] = parcellated[desc][0][rois]

In [23]:
print(smoking_parc.shape)
parcellated['flubatine'].shape

(68,)


(68,)

In [28]:
spins = pd.read_csv(os.path.join(base_path, 'parcellations','spins_hungarian_aparc+aseg_ctx.csv'), header=None)
nspins = spins.values.shape[1]

In [25]:

def load_atlas(base_path):
    """
    Load different parcellation files.
    """    
    atlas= {
        'dk_fsaverage_10k': (
            os.path.join(base_path, 'parcellations', 'atlas-desikankilliany_space-fsaverage_den-10k_hemi-L.label.gii.gz'),
            os.path.join(base_path, 'parcellations', 'atlas-desikankilliany_space-fsaverage_den-10k_hemi-R.label.gii.gz')
    ),
    'dk_fsaverage_164k': (
            os.path.join(base_path, 'parcellations', 'atlas-desikankilliany_space-fsaverage_den-164k_hemi-L.aparc-1.annot'),
            os.path.join(base_path, 'parcellations', 'atlas-desikankilliany_space-fsaverage_den-164k_hemi-R.aparc-1.annot')
    ),
    'dk_mni': os.path.join(base_path, 'parcellations', 'atlas-desikankilliany_space-MNI_res-1mm.nii.gz')
    }
    return (atlas) 

# create parcellaters 
def create_parcellaters(atlas):
    dk_fsaverage_10k = relabel_gifti(atlas['dk_fsaverage_10k'])
    parcellater_fs10k = Parcellater(dk_fsaverage_10k, 'fsaverage')
    
    dk_fsaverage_164k = annot_to_gifti(atlas['dk_fsaverage_164k'])  
    parcellater_fs164k = Parcellater(dk_fsaverage_164k, 'fsaverage')

    parcellater_mni = Parcellater(atlas['dk_mni'], 'MNI152')
    return (parcellater_fs10k, parcellater_fs164k, parcellater_mni) 

    
# load atlas 
atlas = load_atlas(base_path)

In [26]:
def calc_corrs(annotations, parc):
    """
    Calculate correlations & nulls between parcellated annotations & 
    parcellated antipsychotic effects on cortical thickness 
    """

    # initialize
    nulls = dict([])
    corrs = dict([])
    
    # go over annotations 
    for src, desc, space, den in annotations:
        if space == 'MNI152':
            parcellation=atlas['dk_mni']
            
        elif space == 'fsaverage' and den == '164k':
            parcellation=atlas['dk_fsaverage_164k']
            
        elif space == 'fsLR' and den == '164k':
            parcellation=atlas['dk_fsaverage_164k']
            
        elif space == 'fsLR' and den != '164k':
            parcellation=atlas['dk_fsaverage_10k']
        
        # empirical correlation between annotations and parc
        rho = pearsonr(parcellated[desc], parc)[0]
        
        # get 10k rotations 
        rotated = hungarian(data=parcellated[desc], n_perm=10000, spins=spins, parcellation=parcellation) 
        
        # get null 
        n = np.zeros((nspins, ))
        for i in range(nspins):
            n[i] = pearsonr(parc, rotated[:,i])[0]    
        
        # get p-value
        pspin = (1 + sum(abs(n) > abs(rho ))) / (nspins + 1)
    
        # store, multiply by -1 to make more intuitive, because smaller p-value/rho means bigger effect  
        corrs[src+'_'+desc] = ( (-1 * rho, pspin ) )
        nulls[src+'_'+desc] = n
        
    return(nulls, corrs)

In [29]:
nulls_smoking, corrs_smoking = calc_corrs(annotations, smoking_parc)

In [30]:
corrs_smoking

{'hillmer2016_flubatine': (-0.0662468273325731, 0.595940405959404),
 'naganawa2020_lsn3172176': (-0.23934712113084428, 0.05299470052994701),
 'tuominen_feobv': (0.35986820255642477, 0.0022997700229977)}